In [1]:
import pandas as pd
import numpy as np
import email_lead_scoring as els
import h2o
import h2o.automl as automl
import mlflow
import mlflow.h2o as mlflow_h2o
import mlflow.sklearn as mlflow_sklearn

/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages/janitor/utils.py:84: FutureWarning: PandasArray has been renamed NumpyExtensionArray. Use that instead. This alias will be removed in a future version.
  @_expand_grid.register(pd.arrays.PandasArray)


In [2]:
h2o.init()
leads_df = els.read_and_process('x')
leads_df

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 days 21 hours 21 mins
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_jp_7mhsp0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.355 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


replacing tag_aws_webinar with 0
replacing tag_learning_lab with 0
replacing tag_learning_lab_05 with 0
replacing tag_learning_lab_09 with 0
replacing tag_learning_lab_11 with 0
replacing tag_learning_lab_12 with 0
replacing tag_learning_lab_13 with 0
replacing tag_learning_lab_14 with 0
replacing tag_learning_lab_15 with 0
replacing tag_learning_lab_16 with 0
replacing tag_learning_lab_17 with 0
replacing tag_learning_lab_18 with 0
replacing tag_learning_lab_19 with 0
replacing tag_learning_lab_20 with 0
replacing tag_learning_lab_21 with 0
replacing tag_learning_lab_22 with 0
replacing tag_learning_lab_23 with 0
replacing tag_learning_lab_24 with 0
replacing tag_learning_lab_25 with 0
replacing tag_learning_lab_26 with 0
replacing tag_learning_lab_27 with 0
replacing tag_learning_lab_28 with 0
replacing tag_learning_lab_29 with 0
replacing tag_learning_lab_30 with 0
replacing tag_learning_lab_31 with 0
replacing tag_learning_lab_32 with 0
replacing tag_learning_lab_33 with 0
replacin

,mailchimp_id,user_full_name,user_email,member_rating,optin_time,country_code,time_as_subscriber,email_sender,tag_aws_webinar,tag_learning_lab,...,tag_learning_lab_44,tag_learning_lab_45,tag_learning_lab_46,tag_learning_lab_47,tag_time_series_webinar,tag_webinar,tag_webinar_01,tag_webinar_no_degree,tag_webinar_no_degree_02,made_purchase
0,3,Garrick Langworth,garrick.langworth@gmail.com,2,2019-05-22,in,1798,gmail,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1,4,Cordell Dickens,cordell.dickens@gmail.com,4,2018-11-19,other,1982,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,8,Inga Dach,inga.dach@gmail.com,2,2018-11-19,other,1982,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,10,Ferdinand Bergstrom,ferdinand.bergstrom@gmail.com,2,2020-03-20,co,1495,gmail,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,11,Justen Simonis,justen.simonis@gmail.com,2,2020-04-14,other,1470,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19914,33405,Ms. Felicity Moore MD,ms.felicity.moore.md@gmail.com,2,2018-11-18,other,1983,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19915,33406,Shirley Rowe,shirley.rowe@gmail.com,1,2019-03-12,br,1869,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19916,33407,Jarrett Walker-Carroll,jarrett.walkercarroll@gmail.com,2,2019-09-09,in,1688,gmail,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19917,33408,Tanja Herzog,tanja.herzog@gmail.com,2,2019-10-24,other,1643,gmail,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [3]:
all_cols = leads_df.columns.tolist()

y_cols = 'made_purchase'


# remove element from list based on list
def remove_element_from_list(list1, list2):
    for i in list2:
        if i in list1:
            list1.remove(i)
    return list1


x_cols = remove_element_from_list(all_cols,
                                  ['made_purchase', 'mailchimp_id', 'user_full_name', 'user_email', 'optin_time'])
x_cols

['member_rating',
 'country_code',
 'time_as_subscriber',
 'email_sender',
 'tag_aws_webinar',
 'tag_learning_lab',
 'tag_learning_lab_05',
 'tag_learning_lab_09',
 'tag_learning_lab_11',
 'tag_learning_lab_12',
 'tag_learning_lab_13',
 'tag_learning_lab_14',
 'tag_learning_lab_15',
 'tag_learning_lab_16',
 'tag_learning_lab_17',
 'tag_learning_lab_18',
 'tag_learning_lab_19',
 'tag_learning_lab_20',
 'tag_learning_lab_21',
 'tag_learning_lab_22',
 'tag_learning_lab_23',
 'tag_learning_lab_24',
 'tag_learning_lab_25',
 'tag_learning_lab_26',
 'tag_learning_lab_27',
 'tag_learning_lab_28',
 'tag_learning_lab_29',
 'tag_learning_lab_30',
 'tag_learning_lab_31',
 'tag_learning_lab_32',
 'tag_learning_lab_33',
 'tag_learning_lab_34',
 'tag_learning_lab_35',
 'tag_learning_lab_36',
 'tag_learning_lab_37',
 'tag_learning_lab_38',
 'tag_learning_lab_39',
 'tag_learning_lab_40',
 'tag_learning_lab_41',
 'tag_learning_lab_42',
 'tag_learning_lab_43',
 'tag_learning_lab_44',
 'tag_learning_lab_4

In [17]:
leads_h2o = h2o.H2OFrame(leads_df)
leads_h2o['made_purchase'] = leads_h2o['made_purchase'].asfactor()


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


### init mlflow experiment and paths

In [5]:
experiment = "h2o_email_lead_scoring"

In [6]:
import os

module_path2 = os.path.abspath(os.path.join('..'))
module_path2

mlflow.set_tracking_uri('file://' + module_path2 + '/mlruns')

In [7]:
client = mlflow.MlflowClient()

In [8]:
if mlflow.get_experiment_by_name(experiment) is None:
    mlflow.create_experiment(experiment)

In [9]:
experiment_val = mlflow.get_experiment_by_name(experiment)
experiment_val


<Experiment: artifact_location='file:///Users/jp/GitHub/ds-python-2/mlruns/351386619680850278', creation_time=1712906202169, experiment_id='351386619680850278', last_update_time=1712906202169, lifecycle_stage='active', name='h2o_email_lead_scoring', tags={}>

In [11]:
mlflow.set_experiment(experiment_name=experiment) # just to make sure right experiment is set

<Experiment: artifact_location='file:///Users/jp/GitHub/ds-python-2/mlruns/351386619680850278', creation_time=1712906202169, experiment_id='351386619680850278', last_update_time=1712906202169, lifecycle_stage='active', name='h2o_email_lead_scoring', tags={}>

In [13]:
mlflow.start_run() # use previously specified experiment name

<ActiveRun: >

### init h2o modeling

In [15]:
aml = automl.H2OAutoML(nfolds=5, # 
                exclude_algos=['DeepLearning'], # usually used for image classification
                max_runtime_secs=2*60,
                seed=11223
                )

In [18]:
aml.train(x=x_cols, y=y_cols, training_frame=leads_h2o)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/26
# GBM base models (used / total),1/13
# XGBoost base models (used / total),5/10
# DRF base models (used / total),1/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [19]:
aml.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_3_AutoML_2_20240412_155153,0.816497,0.154201,0.287561,0.328693,0.198435,0.0393764
StackedEnsemble_BestOfFamily_3_AutoML_2_20240412_155153,0.815736,0.15553,0.277649,0.3322,0.199636,0.0398547
StackedEnsemble_AllModels_2_AutoML_2_20240412_155153,0.815577,0.155131,0.283145,0.337374,0.199284,0.039714
StackedEnsemble_BestOfFamily_4_AutoML_2_20240412_155153,0.815506,0.155454,0.281689,0.32259,0.199557,0.0398231
StackedEnsemble_AllModels_1_AutoML_2_20240412_155153,0.815469,0.155362,0.27951,0.329078,0.199481,0.0397925
StackedEnsemble_BestOfFamily_2_AutoML_2_20240412_155153,0.814941,0.155626,0.277362,0.325756,0.199669,0.0398677
StackedEnsemble_BestOfFamily_1_AutoML_2_20240412_155153,0.812003,0.15631,0.272969,0.324123,0.199888,0.0399553
XGBoost_grid_1_AutoML_2_20240412_155153_model_6,0.811321,0.156743,0.273837,0.338753,0.200219,0.0400876
XGBoost_1_AutoML_2_20240412_155153,0.810268,0.157126,0.271956,0.318587,0.200324,0.0401298
XGBoost_grid_1_AutoML_2_20240412_155153_model_2,0.809776,0.156086,0.279871,0.331978,0.199246,0.0396989


### log metrics to mlflow

In [20]:
mlflow_h2o.log_model(aml.leader, "h2o_automl_model") # in sklearn, use  mlflow.sklearn_log_model

/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [23]:
mlflow.log_metric("log_loss", aml.leader.logloss())
mlflow.log_metric("auc", aml.leader.auc())

log loss (logarithmic loss ) uses output probability (not just the class), unlike other threshold metrics like accuracy, precision and recall. its also adjusts for sensitivity of confidence, for example, if model is 60% confident, it will be penalized less than if it was 90% confident.

auc score (area under the curve) tells about how much the model is able to distinguish between classes and rank predictions correctly.



### tags

In [24]:
mlflow.set_tag("Source", "H2O_model")

In [26]:
active_run_id = mlflow.active_run().info.run_id

In [27]:
mlflow.set_tag("Run ID", active_run_id)

In [39]:
model_uri = mlflow.get_artifact_uri("model")

In [35]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = 'ALL')
print(lb.head(rows=lb.nrows))

model_id                                                      auc    logloss     aucpr    mean_per_class_error      rmse        mse    training_time_ms    predict_time_per_row_ms  algo
StackedEnsemble_AllModels_3_AutoML_2_20240412_155153     0.816497   0.154201  0.287561                0.328693  0.198435  0.0393764                1215                   0.041911  StackedEnsemble
StackedEnsemble_BestOfFamily_3_AutoML_2_20240412_155153  0.815736   0.15553   0.277649                0.3322    0.199636  0.0398547                 848                   0.025291  StackedEnsemble
StackedEnsemble_AllModels_2_AutoML_2_20240412_155153     0.815577   0.155131  0.283145                0.337374  0.199284  0.039714                 1144                   0.027183  StackedEnsemble
StackedEnsemble_BestOfFamily_4_AutoML_2_20240412_155153  0.815506   0.155454  0.281689                0.32259   0.199557  0.0398231                 603                   0.027208  StackedEnsemble
StackedEnsemble_AllModels_1_Aut

In [36]:
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_3_AutoML_2_20240412_155153,0.816497,0.154201,0.287561,0.328693,0.198435,0.0393764,1215,0.041911,StackedEnsemble
StackedEnsemble_BestOfFamily_3_AutoML_2_20240412_155153,0.815736,0.15553,0.277649,0.3322,0.199636,0.0398547,848,0.025291,StackedEnsemble
StackedEnsemble_AllModels_2_AutoML_2_20240412_155153,0.815577,0.155131,0.283145,0.337374,0.199284,0.039714,1144,0.027183,StackedEnsemble
StackedEnsemble_BestOfFamily_4_AutoML_2_20240412_155153,0.815506,0.155454,0.281689,0.32259,0.199557,0.0398231,603,0.027208,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_2_20240412_155153,0.815469,0.155362,0.27951,0.329078,0.199481,0.0397925,809,0.029973,StackedEnsemble
StackedEnsemble_BestOfFamily_2_AutoML_2_20240412_155153,0.814941,0.155626,0.277362,0.325756,0.199669,0.0398677,696,0.031707,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_2_20240412_155153,0.812003,0.15631,0.272969,0.324123,0.199888,0.0399553,963,0.017533,StackedEnsemble
XGBoost_grid_1_AutoML_2_20240412_155153_model_6,0.811321,0.156743,0.273837,0.338753,0.200219,0.0400876,1087,0.00404,XGBoost
XGBoost_1_AutoML_2_20240412_155153,0.810268,0.157126,0.271956,0.318587,0.200324,0.0401298,1268,0.00526,XGBoost
XGBoost_grid_1_AutoML_2_20240412_155153_model_2,0.809776,0.156086,0.279871,0.331978,0.199246,0.0396989,1398,0.005465,XGBoost


### save all created model to mlflow

In [40]:
model_uri

'file:///Users/jp/GitHub/ds-python-2/mlruns/351386619680850278/472d98646a7e4189bff662d8eb839bd5/artifacts/model'

In [45]:
experiment_id = experiment_val.experiment_id
run_id = mlflow.active_run().info.run_id

In [57]:
x = module_path2 + '/mlruns'
f"{x}/{experiment_id}/{run_id}/artifacts/leaderboard.csv"

'/Users/jp/GitHub/ds-python-2/mlruns/351386619680850278/472d98646a7e4189bff662d8eb839bd5/artifacts/leaderboard.csv'

In [58]:

try:
    lb_path = f"{x}/{experiment_id}/{run_id}/artifacts/leaderboard.csv"
    lb.as_data_frame().to_csv(lb_path, index = False)
    print(f"Leaderboard saved to {lb_path}")
except:
    print("Leaderboard not saved")
    

Leaderboard saved to /Users/jp/GitHub/ds-python-2/mlruns/351386619680850278/472d98646a7e4189bff662d8eb839bd5/artifacts/leaderboard.csv


/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [59]:
mlflow.end_run()

### predictions

In [10]:

logged_model = 'runs:/472d98646a7e4189bff662d8eb839bd5/h2o_automl_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,11 days 21 hours 22 mins
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_jp_7mhsp0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.355 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [11]:

loaded_model.predict(leads_df)

/Users/jp/miniconda3/envs/python_ds2_3.9/lib/python3.9/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict,p0,p1
0,0,0.937816,0.062184
1,0,0.961716,0.038284
2,0,0.988517,0.011483
3,0,0.843694,0.156306
4,0,0.959137,0.040863
...,...,...,...
19914,0,0.988519,0.011481
19915,0,0.986056,0.013944
19916,0,0.986522,0.013478
19917,0,0.983399,0.016601
